In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

TOP_POSTER_LIMIT = 10
DOMAIN_LIMIT = 25
REPLIES_THRESHOLD = 150

df = pd.read_csv("data/latest-articles.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"], unit='s')
df["title"] = df["title"].astype("category")
df["posted_by"] = df["posted_by"].astype("category")
df["source_url"] = df["source_url"].astype("category")

In [2]:
# A thread is considered sourced if it has a URL link.
sourced = df[df.source_url != ""]

# This is not entirely accurate since some vanity posts have links.
vanity = df[df.source_url == ""]

In [3]:
def extract_domain(url) -> str:
    return (
        url.
            replace("www.","").
            replace("http://", "").
            replace("https://", "").
            split("/")[0]
    )


sourced.insert(len(sourced.columns)-1, "domain", (
    sourced.
        source_url.
        apply(extract_domain).
        astype("category")
))

<AxesSubplot:xlabel='count', ylabel='domain'>

In [4]:
# Draws a chart that shows where discussion for a particular thread originated from.
data = sourced[["source_url", "domain"]].drop_duplicates()
top_sourced_domains = data.domain.value_counts().iloc[:DOMAIN_LIMIT].index
sns.countplot(
    y="domain",
    data=data,
    order=top_sourced_domains,
)

<AxesSubplot:xlabel='count', ylabel='posted_by'>

In [5]:
# Draws a chart that shows who posted a topic to be discussed that has a URL source.
data = sourced[["source_url", "posted_by"]].drop_duplicates()
top_sourced_posters = data.posted_by.value_counts().iloc[:TOP_POSTER_LIMIT].index
sns.countplot(
    y="posted_by",
    data=data,
    order=top_sourced_posters,
)

In [6]:
# Comment this if you don't want to see everything.
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Draws a table that shows which freepers posted topics from a sourced link that originated
# from a particular domain.
who_is_posting_from_what = (
    sourced[["posted_by", "source_url", "domain"]].
    query("posted_by in @top_sourced_posters and domain in @top_sourced_domains").
    drop_duplicates().
    groupby(["domain", "posted_by"])
)
who_is_posting_from_what[["domain"]].describe()

domain         \
                                                      count unique   
domain                         posted_by                             
americanthinker.com            Kaslin                    68      1   
                               MtnClimber                90      1   
                               ProfessorGoldiloxx         1      1   
                               SeekAndFind               27      1   
apnews.com                     RandFan                    2      1   
                               conservative98             1      1   
                               nickcarraway               2      1   
breitbart.com                  ProfessorGoldiloxx         3      1   
                               RandFan                    2      1   
                               conservative98            19      1   
                               ChicagoConservative27    163      1   
                               Red Badger                 3      1   
citizenfreepress.com           MtnClimber                 1      1   
                               SeekAndFind                4      1   
                               Red Badger                19      1   
dailyexpose.uk                 ransomnote                48      1   
dailymail.co.uk                SeekAndFind                1      1   
                               RandFan                   10      1   
                               conservative98             4      1   
                               nickcarraway               1      1   
                               Red Badger                 1      1   
foxnews.com                    Kaslin                     1      1   
                               ProfessorGoldiloxx         7      1   
                               SeekAndFind                1      1   
                               conservative98            22      1   
                               nickcarraway               2      1   
                               ChicagoConservative27     40      1   
                               ransomnote                 1      1   
generaldispatch.whatfinger.com ProfessorGoldiloxx        33      1   
lifesitenews.com               SeekAndFind                1      1   
                               ransomnote                 1      1   
msn.com                        SeekAndFind                2      1   
                               RandFan                    1      1   
                               nickcarraway               6      1   
                               ChicagoConservative27      1      1   
                               Red Badger                 2      1   
nationandstate.com             SeekAndFind               19      1   
                               Red Badger                 1      1   
news.yahoo.com                 SeekAndFind                1      1   
                               RandFan                    1      1   
                               conservative98             3      1   
                               nickcarraway               9      1   
newsbusters.org                ProfessorGoldiloxx         1      1   
                               Red Badger                 1      1   
nypost.com                     ProfessorGoldiloxx         4      1   
                               SeekAndFind                1      1   
                               RandFan                    3      1   
                               conservative98             6      1   
                               nickcarraway              41      1   
                               ChicagoConservative27     52      1   
                               Red Badger                 4      1   
pjmedia.com                    ProfessorGoldiloxx         3      1   
                               SeekAndFind               24      1   
redstate.com                   MtnClimber                 1      1   
                               ProfessorGoldiloxx         2    

In [7]:
# Display which posts got the most replies before
# they aged out from the front page.

(sourced[['timestamp', 'title', 'replies']].
     loc[sourced.replies > REPLIES_THRESHOLD].
     groupby('title').
     max().
     sort_values('replies', ascending=False).
     dropna())

,timestamp,replies
title,,
